In [1]:
from func import *
from typing import Callable, Iterable, List, Tuple
import plotly.graph_objects as go

In [2]:
haxby_dataset = datasets.fetch_haxby(subjects= [1,2,3,4,5,6])


In [3]:
# Creating stimuli to category and category to stimuli:
stimuli2category = {
                        'scissors'     : 0,
                        'face'         : 1, 
                        'cat'          : 2,
                        'scrambledpix' : 3,
                        'bottle'       : 4,
                        'chair'        : 5,
                        'shoe'         : 6,
                        'house'        : 7
}

category2stimuli = {category:stimuli for stimuli, category in stimuli2category.items()}

def fetch_haxby_per_subject(subject_id:int = None,standardize:bool = True) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    
        Given the subject id, fetch the haxby data in matrix format.
        
        Arguments:
            - subject_id  (int) : Subject number from [1,6]
            - standardize (bool): If true, masks are standardized
            
        Returns:
            - data (Tuple[np.ndarray, np.ndarray, np.ndarray]) = Original 4-D data, Flattened + Masked Data, Label  
    
    """
        
    # Getting the data file name:
    spatio_temporal_data_path = haxby_dataset.func[subject_id]  
   
    # Getting labels:
    behavioral = pd.read_csv(haxby_dataset.session_target[subject_id], delimiter = ' ')
    
    # Creating conditional categories:
    conditions = behavioral['labels']
    
    # Creating masks for stimuli categories, (ignores rest conditions)
    condition_mask = conditions.isin([*stimuli2category]).tolist()
    
    # Appylying masks to labels (categorical):
    conditions = conditions[condition_mask]
    
    # Creating labels series (numerical):
    categories = np.array([stimuli2category[stimulus] for stimulus in conditions])
    
    # Masking fMRI images: (shape = (40, 64, 64, 864))
    fmri_niimgs = index_img(spatio_temporal_data_path, condition_mask)
    
    # Converting NumPy and transposing to (864, 40, 64, 64):
    numpy_fmri = fmri_niimgs.get_fdata().transpose(3,0,1,2)
    
    masker = NiftiMasker(mask_img=haxby_dataset.mask_vt[subject_id],
                         smoothing_fwhm=4,
                         standardize=standardize,
                         memory='nilearn_cache',
                         memory_level=1)

    masked = masker.fit_transform(fmri_niimgs)
    
    return numpy_fmri,  masked, categories

We mask regions of interest to the original 4D fMRI data and extract it into a numpy matrix. 

In [4]:
data = [fetch_haxby_per_subject(subject_id) for subject_id in range(6)]
fmri_imgs_mat, masks, categories = list(zip(*data))

In [5]:
# set the dir to save plots
explanatory_fMRI_dir = "./images"

In [6]:
from visualizer.plot2D import plot_2d
from visualizer.plot3D import plot_3d 

In [7]:
from utils.timers import timeit
from utils.metrics import accuracy, confusion_matrix, visualize_confusion_matrix
from utils.savers import save, save_obj, load, load_obj
from utils.reproduce import random_seed
from dataset.fetch_data_matrix import fetch_from_haxby


### Unsupervised & Manifold Learning in Human Brain

Functional MRI data are very high-dimensional if one considers all the voxels or surface coordinates acquired with standard imaging parameters. As in our dataset, with the structure of 4D time-series image data, we have a curve of dimensionality problem. In fMRI data, each data point represents an image at a specific point in time, so the dimensionality of the data is determined by the number of voxels (3D coordinates) in each image and the number of time points. Therefore, as we increase the number of voxels in each image or the number of time points, the amount of data required to represent the entire dataset grows exponentially.

This high dimensionality makes it challenging to analyze and visualize the data effectively. Hence, dimension reduction and manifold learning algorithms can reduce the dimensionality of fMRI space by preserving geodesic relations in the lower space representations.

`to do: add labels to the graph`

`figure out 864`

In [79]:
# # For plotting
# import plotly.io as plt_io
# import plotly.graph_objects as go
# import numpy as np



# def plot_3d(component1: np.ndarray,
#             component2 : np.ndarray,
#             component3 :np.ndarray,
#             path:str,
#             y = None,
#             title: str = None) -> None:
    
#     # create a list of unique categories in y
#     unique_categories = np.unique(y)

#     # create a list of traces for each category
#     traces = []
#     for i, category in enumerate(unique_categories):
#         trace = go.Scatter3d(
#             x=component1[y==category],
#             y=component2[y==category],
#             z=component3[y==category],
#             mode='markers',
#             name=str(category),
#             marker=dict(
#                 size=3,
#                 color=y[y==category],
#                 colorscale='Rainbow',
#                 cmin=0,
#                 cmax=len(unique_categories) - 1,
#                 colorbar=dict(
#                     title="Category",
#                     titleside="right",
#                     tickmode="array",
#                     tickvals=[i],
#                     ticktext=[category]
#                 ),
#                 opacity=1,
#                 line_width=1
#             )
#         )
#         traces.append(trace)

#     # create the plot with multiple traces
#     fig = go.Figure(data=traces)

#     # set the layout with legend
#     fig.update_layout(
#         margin=dict(l=50,r=50,b=50,t=50), 
#         width=800, 
#         height=600, 
#         autosize=True,
#         showlegend=True,
#         legend=dict(
#             orientation="h", 
#             yanchor="top", 
#             y=0, 
#             xanchor="center", 
#             x=0.5
#         ),
#         title = title 
#     )

#     # set the template
#     fig.layout.template = 'plotly_dark'

#     # show the plot
#     fig.show()



# def plot_2d(component1: np.ndarray, 
#             component2: np.ndarray, 
#             path: str, y=None,
#             title: str = None) -> None:
#     # Create a list of unique categories
#     categories = np.unique(y)

#     # Create a list of colors for each category


#     # Create a trace for each category
#     traces = []
#     for i, category in enumerate(categories):
#         mask = y == category
#         trace = go.Scatter(
#             x=component1[mask],
#             y=component2[mask],
#             mode='markers',
#             name=str(category),
#             marker=dict(
#                 size=5,
#                 colorscale='Rainbow',
#                 opacity=0.7,
#                 line=dict(width=1)
#             )
#         )
#         traces.append(trace)

#     # Create the figure and add the traces
#     fig = go.Figure(data=traces)

#     # Set the layout
#     fig.update_layout(
#         margin=dict(l=100, r=100, b=100, t=100),
#         width=800,
#         height=600,
#         autosize=True,
#         showlegend=True,
#         legend=dict(
#             orientation="h",
#             yanchor="top",
#             y=1.1,
#             xanchor="center",
#             x=0.5
#         ),
#         title = title 
#     )
#     fig.layout.template = 'plotly_dark'

#     # Show the plot
#     fig.show()



## Dimension Reduction Methods

### PCA

PCA is a popular linear unsupervised dimension reduction algorithm. It finds the principal components of the data, which are the orthogonal linear combinations of the original features that explain the maximum amount of variance in the data. PCA then projects the data onto a lower-dimensional space defined by the principal components, resulting in a lower-dimensional representation of the data.

Here, we apply the PCA to RoI’s of subject 1 and visualize 2D and 3D result.

In [61]:
subject_id = 0

In [80]:
from sklearn.decomposition import PCA
x = masks[subject_id]
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)

principal = pd.DataFrame(data = principalComponents
             ,columns = ['principal component 1',
                         'principal component 2',
                         'principal component 3'])

plot_2d(principalComponents[:, 0],
        principalComponents[:, 1],
        y = categories[subject_id],
        path = os.path.join(explanatory_fMRI_dir, 'pca_2d.png'),
        title="2D PCA Plot"
       )

In [67]:
plot_3d(principalComponents[:, 0],
        principalComponents[:, 1],
        principalComponents[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'pca_3d.png'),
        y = categories[subject_id],
        title="3D PCA Plot")

### LDA Linear Discriminant Analysis

LDA is supervised dimensionality reduction algorithm
and it is a generalization of Fisher’s linear discriminant,
aims to find linear subspace that characterize the original
data space.

In [81]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
x = masks[subject_id]
y = categories[subject_id]

X_LDA = LDA(n_components=3).fit_transform(x,y)

plot_3d(X_LDA[:, 0],
        X_LDA[:, 1],
        X_LDA[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'lda_3d.png'),
        y = categories[subject_id],
        title="3D LDA Plot")

### Independent Component Analysis (ICA)

Independent Component Analysis is a statistical technique for separating a set of observed signals into independent, non-Gaussian components by assuming that the observed signals are linear mixtures of the underlying independent components.

In [43]:
from sklearn.decomposition import FastICA
fast_ica = FastICA(n_components = 3)
ICs = fast_ica.fit_transform(x)


plot_3d(ICs[:, 0],
        ICs[:, 1],
        ICs[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'ica_3d.png'),
        y = categories[subject_id])

### T-Stochastic Neighbor Embedding (t-SNE)

t-SNE converts affinities of data points to probabilities. The affinities in the original space are represented by Gaussian joint probabilities and the affinities in the embedded space are represented by Student’s t-distributions. It minimizes the Kullback-Leibler divergence between the joint probablities of the low-dimensional embedding and the high-dimensional data. t-SNE focuses on the local structure of the data and will tend to extract clustered local groups of samples.

In [76]:
from sklearn.manifold import TSNE
x = masks[subject_id]

tsne = TSNE(random_state = 42,
            n_components=3,
            verbose=0,
            perplexity=40,
            n_iter=400).fit_transform(x)

In [82]:
plot_2d(tsne[:, 0],
        tsne[:, 1],
        path = os.path.join(explanatory_fMRI_dir, 'tsene_2d.png'),
        y = categories[subject_id],
        title="2D t-SNE Plot")

In [83]:
plot_3d(tsne[:, 0],
        tsne[:, 1],
        tsne[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'tsene_3d.png'),
        y = categories[subject_id],
        title="3D t-SNE Plot")

### Uniform Manifold Approximation and Projection (UMAP)

UMAP uses a probabilistic framework that optimizes a low-dimensional embedding to approximate the high-dimensional data, and it emphasizes the preservation of local neighborhoods and connectivity.UMAP is particularly useful for visualizing high-dimensional data in a way that facilitates pattern recognition and data exploration, and it has become a popular alternative to t-SNE and PCA for unsupervised dimensionality reduction.

In [84]:
import umap.umap_ as umap

reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(x)


plot_3d(embedding[:, 0],
        embedding[:, 1],
        embedding[:, 2],
         path = os.path.join(explanatory_fMRI_dir, 'umap_3d.png'),
        y = categories[subject_id],
        title="3D UMAP Plot")

### Mutidimensional Scaling (MDS)

MDS seeks a low-dimensional representation of the data in which the distances respect well the distances in the original high-dimensional space. In general, MDS is a technique used for analyzing similarity or dissimilarity data. It attempts to model similarity or dissimilarity data as distances in a geometric spaces. The data can be ratings of similarity between objects, interaction frequencies of molecules, or trade indices between countries.

In [85]:
from sklearn.manifold import MDS


embedding = MDS(n_components=3)
manifold = embedding.fit_transform(x,categories[subject_id])


plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'mds_3d.png'),
        y = categories[subject_id],
        title="3D MDS Plot")

### ISOMAP

Isometric Mapping is one of the earliest approaches to manifold learning, and it seeks a lower-dimensional embedding which maintains geodesic distances between all points. This ensures that the structure of the data in the lower-dimensional space is as close as possible to the structure of the data in the original high-dimensional space.

In [86]:
from sklearn.manifold import Isomap
x = masks[subject_id]

embedding = Isomap(n_components=3)
manifold = embedding.fit_transform(x)


plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'isomap_3d.png'),
        y = categories[subject_id],
        title="3D ISOMAP Plot")

### Locally Linear Embedding

Locally linear embedding (LLE) seeks a lower-dimensional projection of the data which preserves distances within local neighborhoods. It can be thought of as a series of local Principal Component Analyses which are globally compared to find the best non-linear embedding.

In [87]:
from sklearn.manifold import LocallyLinearEmbedding


embedding = LocallyLinearEmbedding(n_components=3)
manifold = embedding.fit_transform(x,categories[subject_id])


plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'lle_3d.png'),
        y = categories[subject_id],
        title="3D LLE Plot")

### Spectral Embedding
pectral embedding is also non-linear embedding algorithm that forms an affinity matrix and applies spectral decomposition to the laplacian graph.
This technique relies on the basic assumption that the data lies in a low-dimensional manifold in a high-dimensional space.The Spectral Embedding consists of three stages:
1. Weighted Graph Construction

    Transform the raw input data into graph representation using affinity (adjacency) matrix representation. 

2. Graph Laplacian Construction: $L = D^{-\frac{1}{2}}(D-A)D^{-\frac{1}{2}}$
3. Partial Eigenvalue Decomposition: Eigenvalue decomposition is done on graph Laplacian

In [88]:
from sklearn.manifold import SpectralEmbedding

embedding = SpectralEmbedding(n_components=3)
manifold = embedding.fit_transform(x)

plot_3d(manifold[:, 0],
        manifold[:, 1],
        manifold[:, 2],
        path = os.path.join(explanatory_fMRI_dir, 'SpectralEmbedding_3d.png'),
        y = categories[subject_id],
        title="3D Spectral Embedding Plot")

### Reference:

“Laplacian Eigenmaps for Dimensionality Reduction and Data Representation” M. Belkin, P. Niyogi, Neural Computation, June 2003; 15 (6):1373-1396